In [1]:
import plotly
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected = True)
import numpy as np
import ipywidgets as widgets
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt
import json

In [2]:
#define widgets here
co2_ppm = widgets.Dropdown(
    options=['0', '10', '100', '1000'],
    value='10',
    description='CO2[ppm]:',
    disabled=False,
)
options_gases = ['T(K)','Pressure(mbar)','O2(mbar)','N2(ppm)']
#options_gases = ['T(K)','Pressure(mbar)','Water vapor(mbar)', 
#                           'Ozone(ppm)', 'CO2(ppm)','CO2(uatm)','CH4(ppm)','CH4(uatm)']
dropdown_atoms_profiles = widgets.Dropdown(
                    options=options_gases,
                    value='T(K)',
                    description='X-axis data:',
                    disabled=False,
)

In [3]:
dict_atmospheric = {options_gases[0]:'t',options_gases[1]:'p',options_gases[2]:'o2',options_gases[3]:'n2' }

In [4]:
## from: https://stackoverflow.com/questions/22417484/plancks-formula-for-blackbody-spectrum
import scipy as sp
import math
import matplotlib.pyplot as plt
import numpy as np
pi = np.pi
h = 6.626e-34
c = 3.0e+8
k = 1.38e-23

def planck(wav, T):
    a = 2.0*h*pi*c**2
    b = h*c/(wav*k*T)
    intensity = a/ ( (wav**5)*(math.e**b - 1.0) )
    return intensity

wavelengths = np.arange(1e-6, 30e-6, 2e-7) 

# intensity at 220, 240, 260, 280 and 300 K
intensity220 = planck(wavelengths, 220.)/(1e6)
intensity240 = planck(wavelengths, 240.)/(1e6)
intensity260 = planck(wavelengths, 260.)/(1e6)
intensity280 = planck(wavelengths, 280.)/(1e6)
intensity300 = planck(wavelengths, 300.)/(1e6)
#plt.plot(wavelengths*1e6, intensity220, 'r-') 
# plot intensity4000 versus wavelength in nm as a red line
# plt.plot(wavelengths*1e6, intensity240, 'g-') 
# plt.plot(wavelengths*1e6, intensity260, 'b-') 
# plt.plot(wavelengths*1e6, intensity280, 'k-') 
# plt.plot(wavelengths*1e6, intensity300, 'm-')

In [5]:
# load the index json file-- for different co2 conc
with open('toc_files.json','r') as infile:
    co2_dict = json.load(infile)

In [6]:
#build default values for widget 1
dir_name = Path(co2_dict[co2_ppm.value])
pqfile = dir_name / 'rad_spectrum.pq'
df = pd.read_parquet(pqfile)
x_values = df[df.keys()[1]]
scaled_intensity = (df[df.keys()[-1]]*planck(df[df.keys()[1]]*1e-6,300.))/1e6
trace0 = go.Scatter(x=x_values,  y=scaled_intensity, mode="lines", name = 'Model')
bbr_220 = go.Scatter(x=wavelengths*1e6, y=intensity220, mode="lines", name = '220 K')
bbr_240 = go.Scatter(x=wavelengths*1e6, y=intensity240, mode="lines", name = '240 K')
bbr_260 = go.Scatter(x=wavelengths*1e6, y=intensity260, mode="lines", name = '260 K')
bbr_280 = go.Scatter(x=wavelengths*1e6, y=intensity280, mode="lines", name = '280 K')
bbr_300 = go.Scatter(x=wavelengths*1e6, y=intensity300, mode="lines", name = '300 K')

# Now build the figure and define non-default parameters for this figure
g = go.FigureWidget(data=[trace0,bbr_220,bbr_240,bbr_260,bbr_280,bbr_300], 
                    layout=go.Layout(title=dict(text='Radiation spectrum')))
g.layout.width = 600
g.layout.height = 400
g.layout.xaxis.title = 'Wavelength in $\mu$m'
g.layout.yaxis.title = 'Intensity(W/m2 micron)'
g.layout.xaxis.range = [5.,25.]

In [7]:
x1_values = df[df.keys()[1]]
y1_values = df[df.keys()[-3]]
trace1 = go.Scatter(x=x1_values, y=y1_values, mode="lines")

# Now build the figure and define non-default parameters for this figure
g1 = go.FigureWidget(data=[trace1], 
                    layout=go.Layout(title=dict(text='Total radiance spectrum')))
g1.layout.width = 600
g1.layout.height = 400
g1.layout.xaxis.title = 'Wavelength in micrometers'
g1.layout.yaxis.title = 'Total_radiance'
g1.layout.xaxis.range = [5.,25.]

In [8]:
# load the atmospheric profiles
the_dir = co2_dict[co2_ppm.value]
keep_profs = dict()
profs=['mol_prof.pq','aero_prof.pq','o3_prof.pq']
for a_prof in profs:
    the_file = Path(the_dir) / a_prof
    key=the_file.stem
    keep_profs[key] = pd.read_parquet(the_file)

In [9]:
x2_values = keep_profs['mol_prof']['t']
y2_values = keep_profs['mol_prof']['z']
trace2 = go.Scatter(x=x2_values, y=y2_values, mode="lines")

# Now build the figure and define non-default parameters for this figure
g2 = go.FigureWidget(data=[trace2], 
                    layout=go.Layout(title=dict(text='Mol_prof')))
g2.layout.width = 300
g2.layout.height = 500
g2.layout.xaxis.title = dropdown_atoms_profiles.value
g2.layout.yaxis.title = 'Altitude z (km)'
#g2.layout.xaxis.range = [5.,25.]

In [10]:
# plt.plot(keep_profs['mol_prof']['t'], keep_profs['mol_prof']['z'])

# plt.plot(keep_profs['o3_prof']['mol_scat'], keep_profs['o3_prof']['z'])

# plt.plot(keep_profs['o3_prof']['o2'], keep_profs['o3_prof']['z'])

# plt.plot(keep_profs['o3_prof']['p'], keep_profs['aero_prof']['z'])

# plt.plot(keep_profs['o3_prof']['n2'], keep_profs['o3_prof']['z'])

# keep_profs.keys()

In [11]:
def rad_spec(change):
    dir_name = Path(co2_dict[co2_ppm.value])
    pqfile = dir_name / 'rad_spectrum.pq'
    df = pd.read_parquet(pqfile)
    wavelength_um = df[df.keys()[1]]
    tot_transm = df[df.keys()[-1]]
    total_rad = df[df.keys()[-3]]
    scaled_intensity = (tot_transm*planck(df[df.keys()[1]]*1e-6,300.))/1e6
    g.data[0].x = wavelength_um
    g.data[0].y = scaled_intensity
    g.layout.xaxis.title = 'Wavelength in $\mu$m'
    g.layout.xaxis.range = [5.,25.]
    g1.data[0].x = wavelength_um
    g1.data[0].y = total_rad
    g1.layout.xaxis.title = 'Wavelength in $\mu$m'

    
# The next few calls I don't really understand. 
# Presumably I have to look up what the "observe" method is for "widget" objects. 
# It seems as if the "observe" method needs two parameters: 1) the function to call and 2) the "names" parameters. 
# `names="value"` seems to be saying: "pass these parameters with their values into the 'response' function". 
# Or something like that.

co2_ppm.observe(rad_spec, names="value")

In [12]:
def atmos_profile(change):
    the_dir = co2_dict[co2_ppm.value]
    keep_profs = dict()
    profs=['mol_prof.pq','aero_prof.pq','o3_prof.pq']
    for a_prof in profs:
        the_file = Path(the_dir) / a_prof
        key=the_file.stem
        keep_profs[key] = pd.read_parquet(the_file)
    
    x2_values = keep_profs['o3_prof'][dict_atmospheric[dropdown_atoms_profiles.value]]
    y2_values = keep_profs['o3_prof']['z']
    g2.data[0].x = x2_values
    g2.data[0].y = y2_values
    g2.layout.xaxis.title = dropdown_atoms_profiles.value
    g2.layout.yaxis.title = 'Altitude z (km)'

#     plt.legend([220, 240, 260, 280, 300])

dropdown_atoms_profiles.observe(atmos_profile, names="value")

In [13]:
#keep_profs['o3_prof']

,i,z,p,t,n2,cntmslf,mol_scat,n-1,o3,o2
0,1.0,0.0,1013.000,299.7,7.622000e-01,0.0,9.112000e-01,2.621000e-04,0.0,33420.000000
1,2.0,1.0,904.000,293.7,6.321000e-01,0.0,8.298000e-01,2.386000e-04,0.0,29270.000000
2,3.0,2.0,805.000,287.7,5.223000e-01,0.0,7.543000e-01,2.169000e-04,0.0,25620.000000
3,4.0,3.0,715.000,283.7,4.238000e-01,0.0,6.794000e-01,1.954000e-04,0.0,22190.000000
4,5.0,4.0,633.000,277.0,3.484000e-01,0.0,6.160000e-01,1.772000e-04,0.0,19450.000000
5,6.0,5.0,559.000,270.3,2.853000e-01,0.0,5.575000e-01,1.603000e-04,0.0,17030.000000
6,7.0,6.0,492.000,263.6,2.324000e-01,0.0,5.032000e-01,1.447000e-04,0.0,14900.000000
7,8.0,7.0,432.000,257.0,1.885000e-01,0.0,4.531000e-01,1.303000e-04,0.0,13020.000000
8,9.0,8.0,378.000,250.3,1.522000e-01,0.0,4.071000e-01,1.171000e-04,0.0,11380.000000
9,10.0,9.0,329.000,243.6,1.217000e-01,0.0,3.641000e-01,1.047000e-04,0.0,9902.000000


In [14]:
container2 = widgets.HBox([co2_ppm])
container3 = widgets.HBox([dropdown_atoms_profiles])
# Finally, run the dashboard. 
L = widgets.VBox([container2, g])
L1 = widgets.VBox([g1])
L2 = widgets.VBox([container3, g2])
spec = widgets.VBox([L, L1])
widgets.HBox([spec,L2])